# Time-evolution of the Fermi-Hubbard model

This notebook shows how to simulate the one-dimensional Fermi-Hubbard Hamiltonian

$$
H = - J \sum_{j = 1}^{L - 1} c_{j, \sigma}^\dagger c_{j + 1, \sigma} + \text{h.c.} + U \sum_{j} n_{j\uparrow} n_{j\downarrow}
$$

using FQE. Here $j = 1, ..., L$ denotes site/orbital and $\sigma \in \{ \uparrow, \downarrow \}$ denotes spin. We closely follow the [Fermi-Hubbard experiment paper](https://arxiv.org/abs/2010.07965) and the corresponding [ReCirq tutorial](https://quantumai.google/cirq/experiments/fermi_hubbard/experiment_example).

## Setup

In [1]:
import numpy as np

import fqe
from fqe.hamiltonians.diagonal_coulomb import DiagonalCoulomb
from recirq.fermi_hubbard import publication

## Define the Fermi-Hubbard Hamiltonian

In [2]:
"""Get the problem parameters (Hamiltonian, initial state, simulation time)
on a number of different qubit layouts.
"""
# Get publication layouts for a given number of sites.
layouts = publication.rainbow23_layouts(sites_count=8)

# Define problem parameters on each site.
parameters = [
    publication.trapping_instance(
        layout, u=2, dt=0.3, up_particles=2, down_particles=2
    ) 
    for layout in layouts
]

In [3]:
"""Use an example set of problem parameters to simulate with FQE."""
params = parameters[0]
print(params.hamiltonian)

Hamiltonian(sites_count=8, j=1.0, u=2, v=0, local_charge=0, local_spin=0, mu_up=0, mu_down=0)


In [4]:
"""Hamiltonian as an openfermion.DiagonalCoulombHamilontian."""
ham = params.hamiltonian.as_diagonal_coulomb_hamiltonian()
ham

## Convert the OpenFermion Hamiltonian to an FQE Hamiltonian

In [5]:
"""Hamiltonian as an fqe.DiagonalCoulomb Hamiltonian."""
dc_ham = DiagonalCoulomb(ham.one_body + ham.two_body)
dc_ham

## Initialize the wavefunction

In [6]:
norbs = 8
n_elec = 4
sz = 0

fqe_wfn = fqe.Wavefunction([[n_elec, sz, norbs]])
fci_data = fqe_wfn.sector((n_elec, sz))

hf_wf = np.zeros((fci_data.lena(), fci_data.lenb()), dtype=np.complex128)
hf_wf[0, 0] = 1  # right most bit is zero orbital.
fqe_wfn.set_wfn(strategy='from_data',
                raw_data={(n_elec, sz): hf_wf})
fqe_wfn.print_wfn()

Sector N = 4 : S_z = 0
a'00000011'b'00000011' (1+0j)


## Time-evolve the wavefunction

In [7]:
evolved_hf_wfn = fqe_wfn.time_evolve(params.dt, dc_ham)
evolved_hf_wfn.print_wfn()

Sector N = 4 : S_z = 0
a'00000011'b'00000011' (-0.7373937155412454+0.675463180551151j)
